[![Binder](https://mybinder.org/badge_logo.svg)](https://github.com/Sistemas-Multimedia/MRVC/blob/master/src/image_IPP.ipynb)

# testing `image_IPP.py`

This notebook runs performs a modification of the file `image_IPP.py` and compare the RD performance.

In [ ]:
!ln -sf ~/quantization/deadzone_quantizer.py .
!ln -sf ~/quantization/distortion.py .
!ln -sf ~/quantization/information.py .
!ln -sf ~/quantization/midtread_quantizer.py .

### Prepare the sequence

In [ ]:
!rm /tmp/*.png

In [ ]:
#sequence = "bus_cif.y4m"
sequence = "akiyo_cif.y4m"

In [ ]:
N_frames = 5 # Min val 3

In [ ]:
!../sequences/extract_frames.sh -s $sequence -n $N_frames

In [ ]:
!sed -z -i 's/\n#temporal_codec = "image_IPP"\n/\ntemporal_codec = "image_IPP"\n/g' config.py
!sed -z -i 's/\ntemporal_codec = "image_IPP_adaptive"\n/\n#temporal_codec = "image_IPP_adaptive"\n/g' config.py
!sed -z -i 's/\ntemporal_codec = "MP4"\n/\n#temporal_codec = "MP4"\n/g' config.py

In [ ]:
!cat -n config.py | grep "temporal_codec" 

In [ ]:
!cat -n config.py | grep "transform" 

In [ ]:
!cat -n config.py | grep "color" 

In [ ]:
!cat -n config.py | grep "spatial_codec" 

## Option 1
Increase the Q_step for P-frames.

In [ ]:
!sed -z -i 's/\n            #return self.E_codec4_DCT(E_k, prefix, k, Q_step \* 2000)\n/\n            return self.E_codec4_DCT(E_k, prefix, k, Q_step \* 2000)\n/g' image_IPP.py
!sed -z -i 's/\n            return self.E_codec4_DCT(E_k, prefix, k, Q_step)\n/\n            #return self.E_codec4_DCT(E_k, prefix, k, Q_step)\n/g' image_IPP.py

In [ ]:
!cat -n image_IPP.py | grep "return self.E_codec4_DCT"

### Select quantization range

In [ ]:
!sed -z -i 's/\nQ_steps = range(42, 21, -3)\n/\n#Q_steps = range(42, 21, -3)\n/g' config.py
!sed -z -i 's/\n#Q_steps = \[128, 64, 32, 16, 8\]\n/\nQ_steps = \[128, 64, 32, 16, 8\]\n/g' config.py

In [ ]:
!cat -n config.py | grep "Q_steps" 

### Run the codec

In [ ]:
# You can see in real-time the output by running "tail -f /tmp/output"
!python RD_curve.py --first_frame=0 --N_frames=$N_frames 1> >(tee /tmp/output_image_IPP_1 >&1) | grep BPP | grep Average | cat -n

In [ ]:
!grep BPP /tmp/output_image_IPP_1 | grep Average | cut -d ' ' -f 4,9 | tr ' ' '\t'  > /tmp/image_IPP_1.txt

In [ ]:
!cat /tmp/image_IPP_1.txt

In [ ]:
RD_image_IPP_1 = []
with open("/tmp/image_IPP_1.txt", 'r') as f:
    for line in f:
        rate, _distortion = line.split('\t')
        RD_image_IPP_1.append((float(rate), float(_distortion)))

## Option 2
All frames use the same Q_step.

In [ ]:
!sed -z -i 's/\n            return self.E_codec4_DCT(E_k, prefix, k, Q_step \* 2000)\n/\n            #return self.E_codec4_DCT(E_k, prefix, k, Q_step \* 2000)\n/g' image_IPP.py
!sed -z -i 's/\n            #return self.E_codec4_DCT(E_k, prefix, k, Q_step)\n/\n            return self.E_codec4_DCT(E_k, prefix, k, Q_step)\n/g' image_IPP.py

In [ ]:
!cat -n image_IPP.py | grep "return self.E_codec4_DCT"

### Select quantization range

In [ ]:
!sed -z -i 's/\nQ_steps = range(42, 21, -3)\n/\n#Q_steps = range(42, 21, -3)\n/g' config.py
!sed -z -i 's/\n#Q_steps = \[128, 64, 32, 16, 8\]\n/\nQ_steps = \[128, 64, 32, 16, 8\]\n/g' config.py

In [ ]:
!cat -n config.py | grep "Q_steps" 

### Run the codec

In [ ]:
# You can see in real-time the output by running "tail -f /tmp/output"
!python RD_curve.py --first_frame=0 --N_frames=$N_frames 1> >(tee /tmp/output_image_IPP_2 >&1) | grep BPP | grep Average | cat -n

In [ ]:
!grep BPP /tmp/output_image_IPP_2 | grep Average | cut -d ' ' -f 4,9 | tr ' ' '\t'  > /tmp/image_IPP_2.txt

In [ ]:
!cat /tmp/image_IPP_2.txt

In [ ]:
RD_image_IPP_2 = []
with open("/tmp/image_IPP_2.txt", 'r') as f:
    for line in f:
        rate, _distortion = line.split('\t')
        RD_image_IPP_2.append((float(rate), float(_distortion)))

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.axes as ax
import pylab

pylab.figure(dpi=150)
pylab.plot(*zip(*RD_image_IPP_1), label="image_IPP_1")
pylab.plot(*zip(*RD_image_IPP_2), label="image_IPP_2")
pylab.title(f"{sequence}")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("AMSE")
plt.legend(loc='upper right')
pylab.show()